# 2022 Toronto Bike Share Analysis

### Scenairo and the task

We are under a scenario to be working on the marketing analyst team at Bike Share Toronto. The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore, our team wants to understand how casual riders and annual members use the bikes differently. From these insights, our team will design a new marketing strategy to convert casual riders into annual members. We need to present our idea backed up with compelling data insights and professional data visualizations.

## Ask:

We are instructed to analyse the past 12 months data obtained to design marketing strategies aimed at converting casual riders into annual members.

Three questions will guide the future marketing program:
1. How do annual members and casual riders use share bikes differently?
2. Why would casual riders buy annual memberships?
3. How can the company use digital media to influence casual riders to become members?

## Prepare:

Here is all the library we will use in this project.

In [1]:
import pandas as pd
import os
import re
import datetime
import requests
import json

We will use the specific public data set: __[Bike Share Toronto Ridership Data](https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/)__ downloaded and proceed the analyze procress.

All the data is sperated by monthly and there are totally 12 *csv* file downloaded from the above path. We then create two dataset to store the absolute path and the table name.

In [2]:
path = "/Users/michaelfong/Desktop/Bikeshare analysis/bikeshare-ridership-2022"

data_list=[]
tablename_list=[]
for infile in os.listdir(path):
    if os.path.splitext(infile)[1]==".csv":
        data_list.append(os.path.abspath(path)+"/"+infile)
        name=re.findall(r"[\d-]+", os.path.splitext(infile)[0])
        tablename_list.append(name)

data_list.sort()
tablename_list.sort()

print(tablename_list)

[['2022-01'], ['2022-02'], ['2022-03'], ['2022-04'], ['2022-05'], ['2022-06'], ['2022-07'], ['2022-08'], ['2022-09'], ['2022-10'], ['2022-11'], ['2022-12']]


Then we have a function to load all the data inside by the `Panda` module with the `pd.read_csv` command.

In [3]:
def read_csv(data_list, tablename_list):
    total = 0
    for i in range(len(data_list)):
        tablename_list[i] = pd.read_csv(data_list[i])

In [4]:
read_csv(data_list, tablename_list)
print(f"There are total {len(tablename_list)} files read.")

There are total 12 files read.


In [5]:
tablename_list[0].head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,14805109,4335,7334,01/01/2022 00:02,Simcoe St / Wellington St North,7269.0,01/01/2022 01:15,Toronto Eaton Centre (Yonge St),5139,Casual Member
1,14805110,126,7443,01/01/2022 00:02,Dundas St E / George St,7270.0,01/01/2022 00:05,Church St / Dundas St E - SMART,3992,Annual Member
2,14805112,942,7399,01/01/2022 00:04,Lower Jarvis / Queens Quay E,7686.0,01/01/2022 00:19,NaN,361,Annual Member
3,14805113,4256,7334,01/01/2022 00:04,Simcoe St / Wellington St North,7269.0,01/01/2022 01:15,Toronto Eaton Centre (Yonge St),4350,Casual Member
4,14805114,4353,7334,01/01/2022 00:05,Simcoe St / Wellington St North,7038.0,01/01/2022 01:17,Dundas St W / Yonge St,5074,Casual Member


Now, we have 12 dataset loaded and we can start the process for data cleaning.

## Process:

### Change Date into international Standard Format

As notice that the date format for all the dataset is not match the standard `YYYY-MM-DD HH:MM` international standard. Therefore we have a function toperform the transformation by `datetime` library in Python as follow.

In [6]:
def store_date_list(df):
    st_list = []
    et_list = []

    for i in range(len(df)):
        tst = df.iloc[i]["Start Time"]
        st = datetime.datetime.strptime(tst,"%m/%d/%Y %H:%M")
        st_list.append(datetime.date.strftime(st,"%Y-%m-%d %H:%M"))
        tet = df.iloc[i]["End Time"]
        et = datetime.datetime.strptime(tet,"%m/%d/%Y %H:%M")
        et_list.append(datetime.date.strftime(et,"%Y-%m-%d %H:%M"))
        
    for i in range(len(df)):
        df.loc[i, ['Start Time']]=st_list[i]
        df.loc[i, ['End Time']]=et_list[i]

As the size of the data is a little bit large, it takes long time for the processing. Therfore, we perform the transformation for each month separatedly. So, we run the 12 lists as follow:

In [7]:
store_date_list(tablename_list[0])
tablename_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56765 entries, 0 to 56764
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Trip Id             56765 non-null  int64  
 1   Trip  Duration      56765 non-null  int64  
 2   Start Station Id    56765 non-null  int64  
 3   Start Time          56765 non-null  object 
 4   Start Station Name  56195 non-null  object 
 5   End Station Id      56649 non-null  float64
 6   End Time            56765 non-null  object 
 7   End Station Name    56017 non-null  object 
 8   Bike Id             56765 non-null  int64  
 9   User Type           56765 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 4.3+ MB


In [8]:
store_date_list(tablename_list[1])
tablename_list[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67010 entries, 0 to 67009
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Trip Id             67010 non-null  int64  
 1   Trip  Duration      67010 non-null  int64  
 2   Start Station Id    67010 non-null  int64  
 3   Start Time          67010 non-null  object 
 4   Start Station Name  66411 non-null  object 
 5   End Station Id      66959 non-null  float64
 6   End Time            67010 non-null  object 
 7   End Station Name    66276 non-null  object 
 8   Bike Id             67010 non-null  int64  
 9   User Type           67010 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 5.1+ MB


In [9]:
store_date_list(tablename_list[2])
tablename_list[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151938 entries, 0 to 151937
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             151938 non-null  int64  
 1   Trip  Duration      151938 non-null  int64  
 2   Start Station Id    151938 non-null  int64  
 3   Start Time          151938 non-null  object 
 4   Start Station Name  150228 non-null  object 
 5   End Station Id      151892 non-null  float64
 6   End Time            151938 non-null  object 
 7   End Station Name    149981 non-null  object 
 8   Bike Id             151938 non-null  int64  
 9   User Type           151938 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 11.6+ MB


In [10]:
store_date_list(tablename_list[3])
tablename_list[3].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245920 entries, 0 to 245919
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             245920 non-null  int64  
 1   Trip  Duration      245920 non-null  int64  
 2   Start Station Id    245920 non-null  int64  
 3   Start Time          245920 non-null  object 
 4   Start Station Name  241505 non-null  object 
 5   End Station Id      245877 non-null  float64
 6   End Time            245920 non-null  object 
 7   End Station Name    241372 non-null  object 
 8   Bike Id             245920 non-null  int64  
 9   User Type           245920 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 18.8+ MB


In [11]:
store_date_list(tablename_list[4])
tablename_list[4].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504926 entries, 0 to 504925
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             504926 non-null  int64  
 1   Trip  Duration      504926 non-null  int64  
 2   Start Station Id    504926 non-null  int64  
 3   Start Time          504926 non-null  object 
 4   Start Station Name  493119 non-null  object 
 5   End Station Id      504842 non-null  float64
 6   End Time            504926 non-null  object 
 7   End Station Name    492700 non-null  object 
 8   Bike Id             504926 non-null  int64  
 9   User Type           504926 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 38.5+ MB


In [12]:
store_date_list(tablename_list[5])
tablename_list[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605645 entries, 0 to 605644
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             605645 non-null  int64  
 1   Trip  Duration      605645 non-null  int64  
 2   Start Station Id    605645 non-null  int64  
 3   Start Time          605645 non-null  object 
 4   Start Station Name  587693 non-null  object 
 5   End Station Id      605524 non-null  float64
 6   End Time            605645 non-null  object 
 7   End Station Name    587275 non-null  object 
 8   Bike Id             605645 non-null  int64  
 9   User Type           605645 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 46.2+ MB


In [13]:
store_date_list(tablename_list[6])
tablename_list[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680308 entries, 0 to 680307
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             680308 non-null  int64  
 1   Trip  Duration      680308 non-null  int64  
 2   Start Station Id    680308 non-null  int64  
 3   Start Time          680308 non-null  object 
 4   Start Station Name  657662 non-null  object 
 5   End Station Id      680204 non-null  float64
 6   End Time            680308 non-null  object 
 7   End Station Name    657330 non-null  object 
 8   Bike Id             680308 non-null  int64  
 9   User Type           680308 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 51.9+ MB


In [14]:
store_date_list(tablename_list[7])
tablename_list[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705150 entries, 0 to 705149
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             705150 non-null  int64  
 1   Trip  Duration      705150 non-null  int64  
 2   Start Station Id    705150 non-null  int64  
 3   Start Time          705150 non-null  object 
 4   Start Station Name  672386 non-null  object 
 5   End Station Id      705020 non-null  float64
 6   End Time            705150 non-null  object 
 7   End Station Name    672375 non-null  object 
 8   Bike Id             705150 non-null  int64  
 9   User Type           705150 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 53.8+ MB


In [15]:
store_date_list(tablename_list[8])
tablename_list[8].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602817 entries, 0 to 602816
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             602817 non-null  int64  
 1   Trip  Duration      602817 non-null  int64  
 2   Start Station Id    602817 non-null  int64  
 3   Start Time          602817 non-null  object 
 4   Start Station Name  570504 non-null  object 
 5   End Station Id      602667 non-null  float64
 6   End Time            602817 non-null  object 
 7   End Station Name    570461 non-null  object 
 8   Bike Id             602817 non-null  int64  
 9   User Type           602817 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 46.0+ MB


In [16]:
store_date_list(tablename_list[9])
tablename_list[9].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499751 entries, 0 to 499750
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             499751 non-null  int64  
 1   Trip  Duration      499751 non-null  int64  
 2   Start Station Id    499751 non-null  int64  
 3   Start Time          499751 non-null  object 
 4   Start Station Name  471821 non-null  object 
 5   End Station Id      499570 non-null  float64
 6   End Time            499751 non-null  object 
 7   End Station Name    472205 non-null  object 
 8   Bike Id             499751 non-null  int64  
 9   User Type           499751 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 38.1+ MB


In [17]:
store_date_list(tablename_list[10])
tablename_list[10].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320229 entries, 0 to 320228
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             320229 non-null  int64  
 1   Trip  Duration      320229 non-null  int64  
 2   Start Station Id    320229 non-null  int64  
 3   Start Time          320229 non-null  object 
 4   Start Station Name  300542 non-null  object 
 5   End Station Id      320069 non-null  float64
 6   End Time            320229 non-null  object 
 7   End Station Name    300798 non-null  object 
 8   Bike Id             320229 non-null  int64  
 9   User Type           320229 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 24.4+ MB


In [18]:
store_date_list(tablename_list[11])
tablename_list[11].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180010 entries, 0 to 180009
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Trip Id             180010 non-null  int64  
 1   Trip  Duration      180010 non-null  int64  
 2   Start Station Id    180010 non-null  int64  
 3   Start Time          180010 non-null  object 
 4   Start Station Name  168937 non-null  object 
 5   End Station Id      179917 non-null  float64
 6   End Time            180010 non-null  object 
 7   End Station Name    168713 non-null  object 
 8   Bike Id             180010 non-null  int64  
 9   User Type           180010 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 13.7+ MB


After changing the date format, we combine all 12 data list into one as follow:

In [19]:
bikeshare_2022=pd.concat(tablename_list)

In [20]:
bikeshare_2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4620469 entries, 0 to 180009
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Trip Id             int64  
 1   Trip  Duration      int64  
 2   Start Station Id    int64  
 3   Start Time          object 
 4   Start Station Name  object 
 5   End Station Id      float64
 6   End Time            object 
 7   End Station Name    object 
 8   Bike Id             int64  
 9   User Type           object 
dtypes: float64(1), int64(4), object(5)
memory usage: 387.8+ MB


### Data Cleaning 

We now perform the data cleanng as follow. We check the name and Id for the start and end station if there are null values.

In [21]:
bikeshare_2022[bikeshare_2022["Start Station Id"].isna()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type


In [22]:
bikeshare_2022[bikeshare_2022["End Station Id"].isna()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
2355,14807785,0,7048,2022-01-01 17:38,Front St W / Yonge St (Hockey Hall of Fame),NaN,2022-01-01 17:38,NaN,6609,Casual Member
2947,14808557,0,7028,2022-01-02 08:24,Gould St / Mutual St,NaN,2022-01-02 08:24,NaN,6112,Annual Member
3046,14808671,0,7506,2022-01-02 11:30,Berkeley St / Adelaide St E - SMART,NaN,2022-01-02 11:30,NaN,551,Annual Member
3517,14809197,0,7030,2022-01-02 17:01,Bay St / Wellesley St W,NaN,2022-01-02 17:01,NaN,2940,Annual Member
3688,14809384,0,7057,2022-01-02 18:40,Simcoe St / Wellington St South,NaN,2022-01-02 18:40,NaN,6330,Annual Member
...,...,...,...,...,...,...,...,...,...,...
176294,20144421,0,7333,2022-12-30 18:36,King St E / Victoria St,NaN,2022-12-30 18:36,NaN,153,Casual Member
177078,20145403,0,7000,2022-12-31 03:22,Fort York Blvd / Capreol Ct,NaN,2022-12-31 03:22,NaN,2302,Casual Member
179174,20147866,0,7020,2022-12-31 19:54,Phoebe St / Spadina Ave,NaN,2022-12-31 19:54,NaN,1337,Casual Member
179333,20148038,0,7407,2022-12-31 20:46,University Ave / Queen St W,NaN,2022-12-31 20:46,NaN,936,Casual Member


In [23]:
bikeshare_2022[bikeshare_2022["Start Station Name"].isna()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
50,14805164,1245,7684,2022-01-01 00:23,NaN,7274.0,2022-01-01 00:44,Queen's Park Cres E / Grosvenor St - SMART,6839,Annual Member
155,14805280,1158,7683,2022-01-01 01:06,NaN,7002.0,2022-01-01 01:25,St. George St / Bloor St W,5505,Annual Member
177,14805305,136,7684,2022-01-01 01:18,NaN,7067.0,2022-01-01 01:20,Yonge St / Harbour St,2958,Annual Member
194,14805333,1374,7685,2022-01-01 01:33,NaN,7144.0,2022-01-01 01:56,Palmerston Ave / Vermont Ave,4149,Annual Member
243,14805399,284,7687,2022-01-01 02:06,NaN,7205.0,2022-01-01 02:11,Rusholme Park Cres / College St,3308,Annual Member
...,...,...,...,...,...,...,...,...,...,...
179955,20148712,2426,7730,2022-12-31 23:44,NaN,7073.0,2023-01-01 00:25,Spadina Ave / Fort York Blvd,5101,Casual Member
179999,20148769,769,7712,2022-12-31 23:54,NaN,7203.0,2023-01-01 00:07,Bathurst St/Queens Quay(Billy Bishop Airport),6531,Casual Member
180001,20148771,733,7712,2022-12-31 23:55,NaN,7203.0,2023-01-01 00:07,Bathurst St/Queens Quay(Billy Bishop Airport),3872,Casual Member
180004,20148774,1961,7706,2022-12-31 23:56,NaN,7677.0,2023-01-01 00:29,1303 Yonge St - SMART,3076,Annual Member


In [24]:
bikeshare_2022[bikeshare_2022["End Station Name"].isna()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
2,14805112,942,7399,2022-01-01 00:04,Lower Jarvis / Queens Quay E,7686.0,2022-01-01 00:19,NaN,361,Annual Member
159,14805284,3957,7418,2022-01-01 01:07,College Park - Yonge St Entrance,7685.0,2022-01-01 02:13,NaN,182,Casual Member
162,14805287,3894,7418,2022-01-01 01:08,College Park - Yonge St Entrance,7685.0,2022-01-01 02:13,NaN,4171,Casual Member
163,14805288,3887,7418,2022-01-01 01:08,College Park - Yonge St Entrance,7685.0,2022-01-01 02:13,NaN,4871,Casual Member
310,14805489,644,7072,2022-01-01 02:43,Fleet St / Bathurst St,7684.0,2022-01-01 02:54,NaN,6911,Casual Member
...,...,...,...,...,...,...,...,...,...,...
179975,20148733,913,7170,2022-12-31 23:47,Spadina Ave / Willcocks St,7757.0,2023-01-01 00:02,NaN,784,Casual Member
179977,20148735,681,7272,2022-12-31 23:47,Yonge St / Dundonald St - SMART,7713.0,2022-12-31 23:59,NaN,5321,Annual Member
179988,20148746,1012,7271,2022-12-31 23:51,Yonge St / Alexander St - SMART,7683.0,2023-01-01 00:08,NaN,412,Annual Member
179991,20148750,993,7271,2022-12-31 23:51,Yonge St / Alexander St - SMART,7683.0,2023-01-01 00:08,NaN,5548,Annual Member


We then clean the value without station Id.

In [25]:
bikeshare_2022_cleaned=bikeshare_2022[bikeshare_2022["End Station Id"].notnull()]

In [26]:
bikeshare_2022_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4619190 entries, 0 to 180009
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Trip Id             int64  
 1   Trip  Duration      int64  
 2   Start Station Id    int64  
 3   Start Time          object 
 4   Start Station Name  object 
 5   End Station Id      float64
 6   End Time            object 
 7   End Station Name    object 
 8   Bike Id             int64  
 9   User Type           object 
dtypes: float64(1), int64(4), object(5)
memory usage: 387.7+ MB


After that we further delete the column of `Start Station Name` and `End Station Name` as there all lots of null values. We will insert the data with joining the other dataset later.

In [27]:
bikeshare_2022_modified = bikeshare_2022_cleaned.drop(columns = ['Start Station Name', 'End Station Name'], axis=1)

In [28]:
bikeshare_2022_modified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4619190 entries, 0 to 180009
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Trip Id           int64  
 1   Trip  Duration    int64  
 2   Start Station Id  int64  
 3   Start Time        object 
 4   End Station Id    float64
 5   End Time          object 
 6   Bike Id           int64  
 7   User Type         object 
dtypes: float64(1), int64(4), object(3)
memory usage: 317.2+ MB


Then we change the datatype for the `Start Station Id` and `End Station id` in order to perform the merge of dataset below.

In [29]:
bikeshare_2022_modified['Start Station Id'] = bikeshare_2022_modified['Start Station Id'].astype("string")
bikeshare_2022_modified['End Station Id'] = bikeshare_2022_modified['End Station Id'].astype("int")
bikeshare_2022_modified['End Station Id'] = bikeshare_2022_modified['End Station Id'].astype("string")

In [30]:
bikeshare_2022_modified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4619190 entries, 0 to 180009
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Trip Id           int64 
 1   Trip  Duration    int64 
 2   Start Station Id  string
 3   Start Time        object
 4   End Station Id    string
 5   End Time          object
 6   Bike Id           int64 
 7   User Type         object
dtypes: int64(3), object(3), string(2)
memory usage: 317.2+ MB


### Collect Station Data and merge dataset

As mentioned above, we found out that there are lots of null values for `Start Station Name` and `End Station Name`. So, we had deleted the two columns and only left the `Start Station Id` and `End Station Id` columns for index. Now, we obtain the full list of Toronto Bikeshare station Data including the geographic data (Latitude and Longitude for the stations) from the web by `Requests` module and loaded the data as a dataset by `Pandas` and `Json` modules as follow.

In [31]:
col_name = ['station_id', 'name', 'lat', 'lon']
station_id_type = {'station_id': 'string',}

info = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')
stations_information = pd.DataFrame(json.loads(info.content)['data']['stations'])[col_name].astype(station_id_type)

stations_information.head()

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Wellesley Station Green P,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667333,-79.399429
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761
4,7004,University Ave / Elm St,43.656518,-79.389099


In [32]:
stations_information.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   station_id  670 non-null    string 
 1   name        670 non-null    object 
 2   lat         670 non-null    float64
 3   lon         670 non-null    float64
dtypes: float64(2), object(1), string(1)
memory usage: 21.1+ KB


Now we can merge the original `bikeshare_2022` dataset with the new `stations_information` dataset by *inner* join with the station ID as index.

In [33]:
bikeshare_2022_merged=bikeshare_2022_modified.merge(stations_information, how='inner', left_on='Start Station Id', right_on='station_id') \
                     .merge(stations_information, how='inner', left_on='End Station Id', right_on='station_id')

In [34]:
bikeshare_2022_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4160150 entries, 0 to 4160149
Data columns (total 16 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Trip Id           int64  
 1   Trip  Duration    int64  
 2   Start Station Id  string 
 3   Start Time        object 
 4   End Station Id    string 
 5   End Time          object 
 6   Bike Id           int64  
 7   User Type         object 
 8   station_id_x      string 
 9   name_x            object 
 10  lat_x             float64
 11  lon_x             float64
 12  station_id_y      string 
 13  name_y            object 
 14  lat_y             float64
 15  lon_y             float64
dtypes: float64(4), int64(3), object(5), string(4)
memory usage: 539.6+ MB


In [35]:
bikeshare_2022_merged.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,End Station Id,End Time,Bike Id,User Type,station_id_x,name_x,lat_x,lon_x,station_id_y,name_y,lat_y,lon_y
0,14805109,4335,7334,2022-01-01 00:02,7269,2022-01-01 01:15,5139,Casual Member,7334,Simcoe St / Wellington St W North,43.646218,-79.385505,7269,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
1,14805113,4256,7334,2022-01-01 00:04,7269,2022-01-01 01:15,4350,Casual Member,7334,Simcoe St / Wellington St W North,43.646218,-79.385505,7269,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
2,15225111,566,7334,2022-04-10 17:12,7269,2022-04-10 17:22,1310,Annual Member,7334,Simcoe St / Wellington St W North,43.646218,-79.385505,7269,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
3,15511493,418,7334,2022-05-06 17:14,7269,2022-05-06 17:21,3105,Casual Member,7334,Simcoe St / Wellington St W North,43.646218,-79.385505,7269,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
4,15679930,697,7334,2022-05-14 20:27,7269,2022-05-14 20:38,2049,Casual Member,7334,Simcoe St / Wellington St W North,43.646218,-79.385505,7269,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653


As the Id column is repeated, we will delete one of them as below.

In [36]:
bikeshare_2022_merged = bikeshare_2022_merged.drop(columns = ['station_id_x', 'station_id_y'], axis=1)

Then by coparing with the original Id data, we rename the `x` to the `Start` data column name and `y` to the `End` data column name as follow.

In [37]:
rename_index = {
    'name_x':'Start Station Name',
    'lat_x':'Start Station lat',
    'lon_x':'Start Station lon',
    'name_y':'End Station Name',
    'lat_y':'End Station lat',
    'lon_y':'End Station lon',
}

bikeshare_2022_merged=bikeshare_2022_merged.rename(columns = rename_index)

In [38]:
bikeshare_2022_merged.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,End Station Id,End Time,Bike Id,User Type,Start Station Name,Start Station lat,Start Station lon,End Station Name,End Station lat,End Station lon
0,14805109,4335,7334,2022-01-01 00:02,7269,2022-01-01 01:15,5139,Casual Member,Simcoe St / Wellington St W North,43.646218,-79.385505,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
1,14805113,4256,7334,2022-01-01 00:04,7269,2022-01-01 01:15,4350,Casual Member,Simcoe St / Wellington St W North,43.646218,-79.385505,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
2,15225111,566,7334,2022-04-10 17:12,7269,2022-04-10 17:22,1310,Annual Member,Simcoe St / Wellington St W North,43.646218,-79.385505,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
3,15511493,418,7334,2022-05-06 17:14,7269,2022-05-06 17:21,3105,Casual Member,Simcoe St / Wellington St W North,43.646218,-79.385505,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653
4,15679930,697,7334,2022-05-14 20:27,7269,2022-05-14 20:38,2049,Casual Member,Simcoe St / Wellington St W North,43.646218,-79.385505,Toronto Eaton Centre (Yonge St) - SMART,43.655431,-79.380653


Finally, we can output the data to a *csv* file and the can process the analyze and data visualization by **Tableau**.

In [39]:
newpath = "/Users/michaelfong/Desktop/Bikeshare analysis"
bikeshare_2022_merged.to_csv(newpath+"/bikeshare_2022_merged.csv")

## Analyze and Share:

A __[dashboard](https://public.tableau.com/views/TorontoBikeShareAnalysis2022/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link)__ is made by Tableau to show the data analysis result by Tableau.

From the dashboard, we can conlude the following:
1. Casual riders are more than annual members in total. Ridership is higher in summer normally. However, in winter with snow, the rides of annual members were higher than casual riders. For all the other time among the year, casual riders used share bikes more than annual members.
2. Annual members used share bikes more on weekday. Vice versa, casual riders used share bikes more on weekends.
3. Average trip duration were from 500s to 1000s for annual members. However, average trip duration for casual members were longer from 800s to 1500s. This observation is not affected by month.
4. For number of trips accoring to hours, we can observe two peaks on 8 o'clock and 15 o'clock on weekday which is the peak hours to work no matter for annual member and casual member. On weekends, we can observe that there was a higher using rate between 11 o'clock to 20 o'clock for all users.
5. From the top 10 station, we could observe that ridership is concentrated within the downtown.
6. By choosing the annual member or casual member in dashboard, we could observe that annual member's top 10 start and end stations were within downtown among the office area. However, casual member top 10 start stations were nearby the TTC stations and top 10 end stations were nearby waterfront.

## Act:

In order to imporve the data analysis, there are some restrictions for the data that we recommend to improve in the future:

    - missing data for station Id and Bike Id shall be resolved
    - obtain the age group for annual members and casual riders if possible
    - collect the data of bike lane for route patterns 

The following is our summary of findings to answer the three questions:
1. How do annual members and casual riders use share bikes differently?

    Annual members used for travel to work rather than casual riders used for leisure. We can observed that there were still steady number of ride trips even on winter for annual members but less rides for casual riders.
    
    
2. Why would casual riders buy annual memberships?

    As annunal member usually rided for work. It is diffcult to persuade casual riders which used for leisure to by annual memberships as that is no use for them in the winter due to heavy snow. Therefore, we recommend to provide "Half-year membership" within May to October or "Summer pass" from July to August in order to attract more people to join a membership first. After their membership period ended, we could provided special offer or promotion offer for them to join the annual membership afterwards.
    
    
3. How can the company use digital media to influence casual riders to become members?

    There are three menthods we suggested:
    - Install small montiors with solar power supply panels on top used stations and create compelling social media campaigns highlghting the benefits of membership, such as discounts and unlimited access to bikes to pique the interest of casual rider. Every time the casual ride have a rider, they could receive the information form the montiors.
    - Utilize targeted online advertising on platforms like Facebook, Istagram, Youtube and also public transport platform like TTC and Go train to reach potntial members effectively
    - Implement a digital referral program to reward current members for referring new members besides the corportate membership program.

## Thank you for reading my project! 